### import spark 등등

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import * 
from datetime import date, timedelta, datetime
import time

#spark session생성
sc = SparkSession.builder.appName("PysparkExample").config("spark.sql.shuffle.partitions","50").config("spark.driver.maxResultSize","5g").config("spark.sql.execution.arrow.enabled", "true").getOrCreate()

#### file을 읽어 df & table생성

In [3]:
fn='13_202011.dat'
tbl='v'+ fn.replace('.dat','')
print(tbl)
BAT_DATA_HOME='c:/kis/workspace/kis-batch-job/file_home/ccr'
df=sc.read.csv(f"{BAT_DATA_HOME}/btCcrj301/fioBtCcrj301_{fn}",sep="\t", header="true")
df.createOrReplaceTempView(tbl)
print(df.count())
print(df.head())

v13_202011
153
Row(plyno='CA20201000038000', gdcd='CA00101001', cvrcd='CCA00001', cvr_seqno='1', nelp_da_flgcd='03', ins_itm_smccd='CA00003', cnrdt='20201102', ins_st='20201102', ins_clstr='20211102', cvr_ins_st='20201102', cvr_ins_clstr='20211102', cvr_ap_prm='74430', isamt_cd='00001', nelp_days='337', nelp_prm='68718', ce_br_orgcd='A000000', ce_bzp_orgcd='A000002', ce_stfno='kpayins070', cr_usecd='1', rp_pth_flgcd=None, cvr_ad_cr_yn='0', co_obj_as_yn='0', ins_imcd='CA00101', sng_cr_flgcd='03', gn_co_obj_flgcd='01', gn_co_appr_flgcd='01', prdm1_is_yn='1', co_obj_psrt='100', ps_st_nelp_prm='68718', cls_yymm='20201130', sys_mpv_dt=None, chbf_nelp_days='0', chaf_nelp_days='337', chbf_ap_prm='0', chaf_ap_prm='74430', chbf_aprt=None, chaf_aprt=None, chbf_rp_prm=None, chaf_rp_prm=None, chbf_ins_days='0', chaf_ins_days='0', annu_ins_days='365', catcd='22', apldt='20201102', week_yn='N', week_ins_st=None, week_nelp_days='0', nelp_prm_week='0', nelp_prm_gn='68718', ps_st_nelp_prm_week='0', ps_

In [ ]:
print(tbl)
cols=f'plyno,cvrcd,cvr_seqno,trtcd'
cols=f'plyno,cvrcd,cvr_ins_st,cvr_ins_clstr,cvr_ins_st2,cvr_ins_clstr2'
cols=f'NELP_PRM,NELP_PRM_WEEK,NELP_PRM_GN,nelp_days,TRT_AP_CVR_PRMSM'
cols=f' * '
# w='where cast(NELP_DAYS as int) > 0 OR (cast(NELP_DAYS as int) <= 0 AND WEEK_CVRCD = "CCA00192" AND trim(INCM_PRM) != "")'
w=( f'where !(a.NELP_PRM <= 0 AND a.NELP_PRM_WEEK <= 0 \n' 
    f'AND a.NELP_PRM_GN <= 0)'
  )

q=f"select {cols}  from {tbl} {w}"
# q=f"select   from {tbl} {w}"
sc.sql(q).show(100)
print(w)

In [ ]:
q=f'''
  select max(bdin2_pym_ct) from v04_01_dec
   where plyno="CA20201000056000"
   '''
# print(q)
sc.sql(q).show()

In [ ]:
sc.sql('select nelp_days,nelp_prm_gn,nelp_prm_gn*100 from v03 where nelp_prm_gn is not null').show()

In [49]:
q="select sum(decimal(NELP_PRM_GN*1000)) from v13_202011"
# q="select 999900000000*1.1 from v13_202011 limit 1"
sc.sql(q).show()


+--------------------------------------------------------------------------------+
|sum(CAST((CAST(NELP_PRM_GN AS DOUBLE) * CAST(1000 AS DOUBLE)) AS DECIMAL(10,0)))|
+--------------------------------------------------------------------------------+
|                                                                     14598710000|
+--------------------------------------------------------------------------------+



In [22]:
df.groupby('NELP_PRM_GN').sum()

DataFrame[NELP_PRM_GN: string]

In [ ]:
# sc.table('101_13').write.csv('d:/kpayins/ss.txt', sep='\t', header='true', emptyValue='')
df.write.format('com.databricks.spark.csv').option('header','true').save('d:/kpayins/ss.csv')

In [ ]:
df.filter(df["plyno"]=='0311201000070000').show()  #filter

In [ ]:
# datafram조회
cols=['bdins1_ifprm','bdins1_dcprm']
df.select(cols).show(10)  #columns sel

In [ ]:
#when조건
df.select('plyno',when(df.ppdt != '20201102',1).otherwise(0)).show(3)

In [ ]:
df.filter(df["ppdt"]>'20201122').show()  #filter

In [ ]:
#isin
df[df.ppdt.isin('20201102')].show(3)

In [ ]:
df.select('plyno','ppdt',df.plyno.startswith("CA")).show(5)

In [ ]:
df.groupby('ppdt').count().show(2)

In [ ]:
# df.sort(df.ppdt.desc()).show()
df.orderBy(["ppdt","plyno"],ascending=[0,1]).show()

In [ ]:
df.select(['plyno','cvrcd']).show()

In [ ]:
wkDate='20201130'
calc_expr13=f'''

'''

In [ ]:
q=f'select {calc_expr13},INCM_PRM,RL_CVR_ST,NELP_PRM_WEEK from v03T'
sc.sql(q).show()

In [ ]:
calc_expr13

In [ ]:
w='where plyno= "CA20201000070000"'
cols='calc_gb,TRT_AP_CVR_PRMSM,NELP_DAYS,INCM_PRM,CHAF_AP_PRM,trim(SYS_MPV_DT),NELP_PRM_WEEK,TRT_AP_CVR_PRMSM,ANNU_INS_DAYS,RL_CVR_ST,AP_NUMR,AP_DNMT'
q=f'select {calc_expr4},{cols} from v02_1  {w}'
sc.sql(q).show()

In [ ]:
fn='00.dat'
tbl='v'+ fn.replace('.dat','')
print(tbl)
BAT_DATA_HOME='c:/kis/workspace/kis-batch-job/file_home/ccr'
df=sc.read.csv(f"{BAT_DATA_HOME}/btCcrJ301/fioBtCcrJ301_{fn}",sep="\t", header="true")
df.createOrReplaceTempView(tbl)
cols=f'SYS_MPV_DT,week_cvrcd,length(SYS_MPV_DT),trim(dctp_cvrcd),trim(sys_mpv_dt) '
q=f'select {cols},case when trim(sys_mpv_dt)="" then 1 else 2 end dd from {tbl}  '
sc.sql(q).show()

In [ ]:
fn='01.dat'
tbl='v'+ fn.replace('.dat','')
print(tbl)
BAT_DATA_HOME='c:/kis/workspace/kis-batch-job/file_home/ccr'
df=sc.read.csv(f"{BAT_DATA_HOME}/btCcrJ103/fioBtCcrJ103_{fn}",sep="\t", header="true")
df.createOrReplaceTempView(tbl)
cols=f' CO_OBJ_ASNO,case when CO_OBJ_ASNO = "" then "0" else "1" end as co_obj_as_yn'
q=f'select {cols} from {tbl} -- where co_obj_asno=""  '
sc.sql(q).show()

In [ ]:
ss=sc.sql("select add_months(to_date(concat('202011','01'),'yyyyMMdd'),-1) as bef_mon")

In [ ]:
print(ss.select("bef_mon").collect())
# type(ss)

In [ ]:

def get_Before_Mon(params):
    try:
        if len(params)==6:
            params=params+'01'
        dt= datetime.strptime(params,'%Y%m%d').replace(day=1).date()- timedelta(days=1)
        return dt.strftime('%Y%m')
    except  Exception as e: 
            print('param error ', e)
            
get_Before_Mon('202011')

In [ ]:
q="select to_date('20201101','yyyyMMdd') as dt"
a=sc.sql(q)

In [ ]:
sc.sql('select * from tt').show()

In [ ]:
import numpy as np
start_list=np.cumsum([1]+s_dict['width']).tolist()[:-1]

In [4]:
## fixed length file을 읽어 dataframe return
## fld_format은 'plyno:17, name:40' 처럼 ,와 :로 구분하여 전달
def read_fld_file(spark,input_file,fld_format,encoding='utf-8'):
    try:
        cName=[]; cLen=[];
        for s in fld_format.split(','):
            cName.append(s.split(':')[0])
            cLen.append(int(s.split(':')[1]))
        s_dict={"columns":cName,"width":cLen}
#         start_list=np.cumsum([1]+s_dict['width']).tolist()[:-1]
# np대체로직        
        total=1
        start_list=[1]
        for x in cLen:
            total += x
            start_list.append(total)
        start_list = start_list[:-1]  
        
        if encoding=="euc-kr":
            euc_data=open(input_file,'r',encoding='euc-kr').read()
            with open(input_file+'.utf','w',encoding="utf-8") as f:
                  f.write(euc_data)
            df=spark.read.text(input_file+'.utf')
        else:
            df=spark.read.text(input_file)

        df=df.select(
           [df.value.substr(
               start_list[i],s_dict['width'][i]
            ).alias(s_dict['columns'][i]) for i in range(len(start_list))
           ]
        )
        return df
    except  Exception as e: 
            print('file read error---> ', e)
            return None

In [12]:
path="c:/kis/workspace/kis-batch-job/file_home/ccr/btCcrM401/fioBtCcrM401_01.dat.fld"
fld_format='id:85,name:37,erc:2,fil:8,dsc:13,fil2:77,crnm:29,fil3:697'
read_fld_file(sc,path,fld_format,"euc-kr").createOrReplaceTempView('tt')

In [13]:
# sc.sql('select substr(id,20,21) from tt').show()
sc.sql('select crnm from tt').show()

+---------------------+
|                 crnm|
+---------------------+
|43가1374          ...|
|43가1374          ...|
|43가1374          ...|
|43가1374          ...|
|43가1374          ...|
|43가1374          ...|
|43가1374          ...|
|43가1374          ...|
|123가1234         ...|
|123가1234         ...|
|213가1234         ...|
|213가1234         ...|
|33나5555          ...|
|33나5555          ...|
|33나5555          ...|
|33나5555          ...|
|33나5555          ...|
|33나5555          ...|
|33나5555          ...|
|33나5555          ...|
+---------------------+
only showing top 20 rows



In [ ]:
path="c:/kis/workspace/kis-batch-job/file_home/ccr/btCcrM401/fioBtCcrM401_01.dat.fld"
# lis_euc=open(path,'r',encoding='euc-kr').readlines()
# with open(path+'_utf','w',encoding="utf-8") as f:
#      f.write('\n'.join( val for val in lis_euc))
lis_euc=open(path,'r',encoding='euc-kr').read()
unicode(lis_euc,'euc-kr').encode('utf-8')

# type(f)
# f.write(encoding='utf-8')
# unicode(f,'euc-kr').encode('utf-8')

In [15]:
cName=[]
cLen=[]
for s in fld_format.split(','):
#     cName.append(s.split(':')[0])
#     cLen.append(int(s.split(':')[1]))
    cName,cLen=s.split(':')

In [17]:
cName
cLen

'697'

In [ ]:
def accumu(lis):
    total=0
    for x in lis:
        total+=x
        yield total


In [ ]:
total=1
nlis=[1]
lis=[33,2,3]
for x in lis:
    total+=x
    nlis.append(total)
#     yield total
nlis[:-1]